In [72]:
# system imports
import os
import sys

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz


# misc
import warnings

import glob

##
import pytz
import datetime as dt
import math
import seaborn as sns

import pickle

In [73]:

#style settings
sns.set(style='whitegrid', rc={'axes.facecolor': 'white'})

#sns.set_style({'font.family':'Arial', 'font.serif':'Times New Roman'})

## Time Domain analysis

In [74]:
def timedomain(rr):
    results = {}

    hr = 60000/rr
    
    results['Mean RR (ms)'] = np.mean(rr)
    results['STD RR/SDNN (ms)'] = np.std(rr)
    results['Mean HR (Kubios\' style) (beats/min)'] = 60000/np.mean(rr)
    results['Mean HR (beats/min)'] = np.mean(hr)
    results['STD HR (beats/min)'] = np.std(hr)
    results['Min HR (beats/min)'] = np.min(hr)
    results['Max HR (beats/min)'] = np.max(hr)
    results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
    results['NNxx'] = np.sum(np.abs(np.diff(rr)) > 50)*1
    results['pNNxx (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)
    return results

## Related functions

In [138]:
def min_max_norm(all_rr_one_list, all_rr_sublist):
    
    mean_norm_list =[]
    std_norm_list = []
    
    mean_norm_bpm_list = []
    std_norm_bpm_list = []
    
    all_rr_one_list = np.array(all_rr_one_list)
    max_val = np.max(all_rr_one_list)
    min_val =np.min(all_rr_one_list)
    
    range_list = max_val-min_val
    
    ####-------beats per minutes--------
    bpm_one_list = 60000/all_rr_one_list
    
    max_bpm = np.max(bpm_one_list)
    
    min_bpm = np.min(bpm_one_list)
    
    range_bpm =  max_bpm - min_bpm
    
    
    for sel_list in all_rr_sublist:
        
        sel_list_np = np.array(sel_list)
        
        
        
        sel_lis_np_sub = sel_list_np - min_val
        
        sel_lis_norm= sel_lis_np_sub/range_list
        
        mean_norm = np.mean(sel_lis_norm)
        mean_norm_list.append(mean_norm)
        
        std_norm  = np.std(sel_lis_norm)   
        std_norm_list.append(std_norm)
        
        ##------------for normalized BPM
        
        sel_list_bpm = 60000/sel_list_np
        
        
        
        sel_bpm_min_sub = sel_list_bpm - min_bpm
        
        sel_bpm_list_norm =  sel_bpm_min_sub/ range_bpm
        
        mean_norm_bpm = np.mean(sel_bpm_list_norm)
        
        std_norm_bpm = np.std(sel_bpm_list_norm)
        
        mean_norm_bpm_list.append(mean_norm_bpm)
        
        std_norm_bpm_list.append(std_norm_bpm)
        
        
        
        
        #print(mean_norm)
        #print(std_norm)
                
        
    return mean_norm_list, std_norm_list, mean_norm_bpm_list, std_norm_bpm_list

In [169]:
def from_rr_to_time_freq_anal(selected_path):
                 
    with open(selected_path, 'rb') as f:
        ibi_data = pickle.load(f)
                    
                    
    subject_id = ibi_data["subject_id"]
    event_interst = ibi_data["event_interst"]
    all_rr_one_list = ibi_data["all_rr_one_list"]
    all_rr_sublists = ibi_data["all_rr_sublists"]
            
    max_rr = np.max(all_rr_one_list)
    #print(max_rr)
    min_rr = np.min(all_rr_one_list)
    #print(min_rr)
    range_rr = max_rr - min_rr    
            
    mean_rr_cumm_list = []
    std_rr_cumm_list  = []
    mean_hr_list = []
    std_hr_list = []
            
    for index, selected_event in enumerate(event_interst):
        sel_rr = all_rr_sublists[index]
                
        #### heart rate from RR ineterval
        hr_from_rr = 60000/sel_rr
        mean_hr = np.mean(hr_from_rr)
        std_hr = np.std(hr_from_rr)
        mean_hr_list.append(mean_hr)
        std_hr_list.append(std_hr)
                
        ##### mean and std rr
        mean_rr = np.mean(sel_rr)     
        std_rr = np.std(sel_rr)
        mean_rr_cumm_list.append(mean_rr)
        std_rr_cumm_list.append(std_rr)
                
                
    mean_norm_hrv_list, std_norm_hrv_list,  mean_norm_bpm_list, std_norm_bpm_list = min_max_norm(all_rr_one_list, all_rr_sublists)
            
    export_dict= {"subject_id":subject_id , 'event_interst': event_interst, "mean_bpm": mean_hr_list, "std_bpm":std_hr_list , 
                              "mean_ibi":mean_rr_cumm_list , "std_ibi": std_rr_cumm_list , "norm_mean_ibi":mean_norm_hrv_list,  "norm_std_ibi": std_norm_hrv_list,
                             "norm_mean_bpm":mean_norm_bpm_list, "norm_std_bpm":  std_norm_bpm_list}
            
            
    return export_dict

In [170]:
selected_file = sk_hr_files_list[0]

In [173]:
selected_file

'/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_filtered_pkl/VP015_161123_DigiRelax_Experiment_2023-11-16_09h47.53.124.pkl'

In [171]:
exported_dict=from_rr_to_time_freq_anal(selected_file)

In [172]:
exported_dict

{'subject_id': 'VP015_161123_DigiRelax_Experiment_2023-11-16_09h47.53.124',
 'event_interst': ['baseline_instruction_ts 3 min',
  'tsst_prep_ts 5 min',
  'tsst_pres_ts 5 min',
  'tsst_pres_ts 10 min',
  'relaxation_prep_ts 5 min',
  'relaxation_prep_ts 10 min',
  'relaxation_prep_ts 15 min'],
 'mean_bpm': [71.82353586427526,
  89.26859676839898,
  107.19791932261047,
  116.56401945280167,
  76.52411173784864,
  77.9562122164734,
  76.16484943767374],
 'std_bpm': [2.644015469173672,
  6.96498915311769,
  9.303676103774013,
  10.106206369567028,
  3.960577392473036,
  3.2354384160594285,
  3.7971049539345616],
 'mean_ibi': [836.5033586448598,
  676.1701074660633,
  564.176245210728,
  518.9494680851063,
  786.186922823219,
  770.9948320413437,
  789.7238756613757],
 'std_ibi': [30.521066136292898,
  52.08195534511415,
  51.721160573459585,
  48.91627939946638,
  41.06117741904236,
  32.13742182190513,
  39.38009737488015],
 'norm_mean_ibi': [0.8583601664965054,
  0.5134415757253128,
  0.

In [158]:
pd.DataFrame.from_dict(exported_dict)

,subject_id,event_interst,mean_bpm,std_bpm,mean_ibi,std_ibi,norm_mean_ibi,norm_std_ibi,norm_mean_bpm,norm_std_bpm
0,VP014_151123_DigiRelax_Experiment_2023-11-15_1...,baseline_instruction_ts 3 min,81.630292,4.732814,737.490315,42.726737,0.575590,0.088927,0.269374,0.071240
1,VP014_151123_DigiRelax_Experiment_2023-11-15_1...,tsst_prep_ts 5 min,89.524221,9.477690,677.677317,71.365101,0.451101,0.148532,0.388195,0.142661
2,VP014_151123_DigiRelax_Experiment_2023-11-15_1...,tsst_pres_ts 5 min,88.317014,8.194845,685.323791,64.698946,0.467015,0.134658,0.370024,0.123351
3,VP014_151123_DigiRelax_Experiment_2023-11-15_1...,tsst_pres_ts 10 min,85.710671,8.235093,706.358712,66.438333,0.510795,0.138278,0.330793,0.123957
4,VP014_151123_DigiRelax_Experiment_2023-11-15_1...,relaxation_prep_ts 5 min,78.887730,5.684148,764.501700,54.899274,0.631808,0.114262,0.228092,0.085559
5,VP014_151123_DigiRelax_Experiment_2023-11-15_1...,relaxation_prep_ts 10 min,78.196993,4.337687,769.683442,43.230559,0.642593,0.089976,0.217695,0.065292
6,VP014_151123_DigiRelax_Experiment_2023-11-15_1...,relaxation_prep_ts 15 min,78.419445,4.660402,767.835514,45.893662,0.638747,0.095519,0.221043,0.070150


In [139]:
def get_non_nan_values_cols(p_df, col_name):
    
    slic_ind=p_df[col_name].isnull()
    list_ne=slic_ind.tolist()
    false_indices = [i for i in range(len(list_ne)) if not list_ne[i]]
    
    list_col_interest=p_df[col_name].to_list()
    
    non_values_list = []
    
    for false_ind in false_indices:
        non_val = list_col_interest[false_ind]
        non_values_list.append(non_val)
    
    return non_values_list


def get_file_names_from_ind_list(files_list, inds_list):
    
    selected_files_list=[files_list[sel_index] for sel_index in inds_list]
    
    return selected_files_list


def from_rr_to_time_freq_anal(sk_hr_files_list, condition_files):

    #mean_sk_acc=[]
    #std_sk_acc = []

    #mean_hr_acc=[]
    #std_hr_acc=[]
    
    result_dict ={"file_name":[]}
    
    

    for selected_path in sk_hr_files_list:
        dir_path, file_name=os.path.split(selected_path)
        sub_id = file_name[0:12]
        result_dict["file_name"].append(sub_id)
        #print(sub_id)
        for cond_file in condition_files:
            if sub_id==cond_file:
                print(selected_path)
                
                with open(selected_path, 'rb') as f:
                    ibi_data = pickle.load(f)
                    
                #sk_hr_df = pd.read_csv(selected_path)
                
                subject_id = ibi_data["subject_id"]
                event_interst = ibi_data["event_interst"]
                
                all_rr_one_list = ibi_data["all_rr_one_list"]
                
                all_rr_sublists = ibi_data["all_rr_sublists"]
                
                max_rr = np.max(all_rr_one_list)
                #print(max_rr)
                min_rr = np.min(all_rr_one_list)
                #print(min_rr)
                
                range_rr = max_rr - min_rr
                
                mean_rr_cumm_list = []
                std_rr_cumm_list  = []
                
                mean_hr_list = []
                std_hr_list = []
                
                
                
                for index, selected_event in enumerate(event_interst):
                    
                    
                    #print(selected_event)
                    sel_rr = all_rr_sublists[index]
                    hr_from_rr = 60000/sel_rr
                    
                    
                    mean_hr = np.mean(hr_from_rr)
                    
                    std_hr = np.std(hr_from_rr)
                    
                    mean_hr_list.append(mean_hr)
                    std_hr_list.append(std_hr)
                    
                    mean_rr = np.mean(sel_rr) 
                    
                    std_rr = np.std(sel_rr)
                    
                    
                    mean_rr_cumm_list.append(mean_rr)
                    
                    std_rr_cumm_list.append(std_rr)
                    
                    
                
                    
                    
                mean_norm_hrv_list, std_norm_hrv_list,  mean_norm_bpm_list, std_norm_bpm_list = min_max_norm(all_rr_one_list, all_rr_sublists)
                
                export_dict= {"subject_id":sub_id , 'event_interst': event_interst, "mean_bpm": mean_hr_list, "std_bpm":std_hr_list , 
                              "mean_ibi":mean_rr_cumm_list , "std_ibi": std_rr_cumm_list , "norm_mean_ibi":mean_norm_hrv_list,  "norm_std_ibi": std_norm_hrv_list,
                             "norm_mean_bpm":mean_norm_bpm_list, "norm_std_bpm":  std_norm_bpm_list}
                
                
                
                

    return export_dict
                    
                    
                    
                    
                    
                    
                    
                    #scaled_
                    #print(np.mean(hr_from_rr))
                    
                    #print(60000/sel_rr)
                    #min_sub = sel_rr - min_rr
                   # scaled_data = min_sub/range_rr
                    
                    #print(scaled_data)
                    
                    
                    #print(results)
                    
                    
                
                
                
                
                
                

                #mean_sk = sk_hr_df[mean_sk_col].values
                #std_sk = sk_hr_df[std_sk_col].values
                #mean_hr =sk_hr_df[mean_hr_col].values
                #std_hr =sk_hr_df[std_hr_col].values


                #mean_sk_acc.append(mean_sk)
                #std_sk_acc.append(std_sk)
                #mean_hr_acc.append(mean_hr)
                #std_hr_acc.append(std_hr)



    #mean_mean_sk=np.mean(np.array(mean_sk_acc), axis=0) 
    #std_mean_sk = np.std(np.array(std_sk_acc), axis=0)
    #std_mean_sk = np.mean(np.array(std_sk_acc), axis=0)

    #mean_mean_hr=np.mean(np.array(mean_hr_acc), axis=0) 
    #std_mean_hr = np.std(np.array(std_hr_acc), axis=0)
    #std_mean_hr = np.mean(np.array(std_hr_acc), axis=0)
    
    #selected_events = sk_hr_df[event_interst_col].to_list()
    
    #extracted_data={"selected_event": selected_events, "hr_mean": mean_mean_hr, "hr_std": std_mean_hr, "sk_mean": mean_mean_sk, "sk_std":std_mean_sk}
    
    #return extracted_data


In [164]:
#sub_data_path = "/home/muhammad/Desktop/Datasets/data_sony_digiRelax/subjective_data/subjective_data.xlsx"
sub_data_path ="/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/subjective_data/DigiR_subj_final_250124.xlsx"
#hrv_data_dir = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp3/results_22sub/hrv_analysis"

#hrv_data_dir = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_pkl"

hrv_data_dir = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_filtered_pkl"


In [165]:
sk_hr_files_list = glob.glob(hrv_data_dir+ "/*.pkl")

In [142]:
sub_data_df=pd.read_excel(sub_data_path)
file_col = "subject"
cond_col ="condition"

file_list = get_non_nan_values_cols(sub_data_df,file_col)
cond_list = get_non_nan_values_cols(sub_data_df, cond_col)

cond_set=set(cond_list)
unique_conds_list= list(cond_set)

In [143]:
print(unique_conds_list)

['2D_no_scent', '2D_scent', 'VR_no_scent', 'VR_scent']


In [162]:
no_scent_2d_inds=np.where(np.array(cond_list)=="2D_no_scent")
no_scent_2d_inds = list(no_scent_2d_inds[0])
no_scent_2d_files =get_file_names_from_ind_list(file_list, no_scent_2d_inds)

scent_2d_inds=np.where(np.array(cond_list)=="2D_scent")
scent_2d_inds=list(scent_2d_inds[0])
scent_2d_files=get_file_names_from_ind_list(file_list, scent_2d_inds)

no_scent_vr_inds = np.where(np.array(cond_list)=="VR_no_scent")
no_scent_vr_inds=list(no_scent_vr_inds[0])
no_scent_vr_files=get_file_names_from_ind_list(file_list, no_scent_vr_inds)

scent_vr_inds =np.where(np.array(cond_list)=="VR_scent")
scent_vr_inds =list(scent_vr_inds[0])
scent_vr_files=get_file_names_from_ind_list(file_list, scent_vr_inds)

In [145]:
print("Total scent vr files:" + str(len(scent_vr_files)))
print("Total scent 2d files:" + str(len(scent_2d_files)))
print("Total non-scent vr files:" + str(len(no_scent_vr_files)))
print("Total non-scent 2d files:" + str(len(no_scent_2d_files)))


Total scent vr files:18
Total scent 2d files:18
Total non-scent vr files:20
Total non-scent 2d files:20


In [146]:
exported_dict=from_rr_to_time_freq_anal(sk_hr_files_list, scent_2d_files)

/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_filtered_pkl/VP075_180124_DigiRelax_Experiment_2024-01-18_09h54.07.734.pkl
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_filtered_pkl/VP072_120124_DigiRelax_Experiment_2024-01-12_14h45.23.547.pkl
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_filtered_pkl/VP060_191223_DigiRelax_Experiment_2023-12-19_09h48.26.679.pkl
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_filtered_pkl/VP041_061223_DigiRelax_Experiment_2023-12-06_15h50.23.237.pkl
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_filtered_pkl/VP039_061223_DigiRelax_Experiment_2023-12-06_09h56.18.635.pkl
/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_filtered_pkl/VP063_040124_DigiRelax_Experiment_2024-01-04_14h59.12.423.pkl
/hom

In [147]:
pd.DataFrame.from_dict(exported_dict)

,subject_id,event_interst,mean_bpm,std_bpm,mean_ibi,std_ibi,norm_mean_ibi,norm_std_ibi,norm_mean_bpm,norm_std_bpm
0,VP056_151223,baseline_instruction_ts 3 min,73.128558,3.776848,822.663484,42.484114,0.470314,0.065915,0.337554,0.059064
1,VP056_151223,tsst_prep_ts 5 min,84.059879,7.626368,719.759270,66.463422,0.310657,0.103119,0.508503,0.119264
2,VP056_151223,tsst_pres_ts 5 min,92.485928,8.444947,654.154908,59.569262,0.208871,0.092423,0.640273,0.132066
3,VP056_151223,tsst_pres_ts 10 min,84.360297,6.448043,715.356504,54.139296,0.303826,0.083998,0.513201,0.100837
4,VP056_151223,relaxation_prep_ts 5 min,64.057828,6.246293,945.449780,90.709817,0.660819,0.140738,0.195702,0.097682
5,VP056_151223,relaxation_prep_ts 10 min,64.628427,5.471633,935.146392,80.224860,0.644833,0.124470,0.204626,0.085568
6,VP056_151223,relaxation_prep_ts 15 min,63.703270,4.285801,946.145999,63.803409,0.661899,0.098992,0.190158,0.067023


In [148]:
import pandas as pd
pd.read_csv("/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/normalized_skin_conductance/normalized_skin_conductance_data/Vp046_081223_DigiRelax_Experiment_2023-12-08_14h49.59.665.csv")

,Unnamed: 0,subject_id,event_interst,mean_sk_cond,std_sk_cond,mean_hr,std_hr
0,0,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,baseline_instruction_ts 3 min,0.179805,0.022457,0.464572,0.039395
1,1,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,tsst_prep_ts 5 min,0.551283,0.092829,0.623152,0.090915
2,2,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,tsst_pres_ts 5 min,0.796113,0.067266,0.861359,0.052378
3,3,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,tsst_pres_ts 10 min,0.832871,0.037005,0.913835,0.034301
4,4,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,relaxation_prep_ts 5 min,0.386072,0.034677,0.498219,0.047131
5,5,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,relaxation_prep_ts 10 min,0.321762,0.017928,0.493214,0.038728
6,6,Vp046_081223_DigiRelax_Experiment_2023-12-08_1...,relaxation_prep_ts 15 min,0.311595,0.033949,0.499834,0.042826


In [91]:
result_dict ={"file_name":[]}

In [93]:
result_dict["file_name"].append("saif")

In [94]:
result_dict

{'file_name': ['saif']}

In [96]:
result_dict["sub_id"].append("1")

KeyError: 'sub_id'